In [3]:
!pip install pandas requests openpyxl

In [21]:
import pandas as pd

# 输入文件路径
input_file_path = r"D:/文档/WeChat Files/WeChat Files/wxid_ujzkebqpjc3v22/FileStorage/File/2024-11/处理后的订单(3).xlsx"

# 输出文件路径
output_directory = r"D:\课程\数据分析\租车"
output_file_path = f"{output_directory}/提取后的11月订单数据.xlsx"

# 需要提取的列名
columns_to_extract = [
    "承租人", "网络平台订单号", "品牌车型", "取车地址", "POI类别", 
    "取车地址类别", "还车地址", "租期", "日租金", "总租金", "渠道"
]

# 读取 Excel 文件
df = pd.read_excel(input_file_path)

# 检查是否包含所有需要提取的列
missing_columns = [col for col in columns_to_extract if col not in df.columns]
if missing_columns:
    raise ValueError(f"以下列在文件中未找到：{missing_columns}")

# 提取需要的列
extracted_df = df[columns_to_extract]

# 保存为新的 Excel 文件
extracted_df.to_excel(output_file_path, index=False)

print(f"数据提取完成，文件已保存到 {output_file_path}")

数据提取完成，文件已保存到 D:\课程\数据分析\租车/提取后的11月订单数据.xlsx


In [41]:
import pandas as pd
import requests
import time

# 高德地图 API 密钥
API_KEY = "ace4cdbddb2061c8b5684b043aa1ed8e"

# 读取 Excel 文件
file_path = "D:/文档/WeChat Files/WeChat Files/wxid_ujzkebqpjc3v22/FileStorage/File/2024-11/20241116153629_订单(3).xlsx"
df = pd.read_excel(file_path)

# 定义函数获取经纬度
def geocode_address(address, api_key):
    url = f'https://restapi.amap.com/v3/geocode/geo?address={address}&key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        if result['status'] == '1' and result.get('geocodes'):
            return result['geocodes'][0]['location']  # 返回经纬度
    return None

# 定义函数获取POI类别
def get_poi_category(location, api_key):
    if location:
        lat, lon = location.split(',')
        url = f'https://restapi.amap.com/v3/place/around?location={lon},{lat}&key={api_key}&radius=1000'
        response = requests.get(url)
        if response.status_code == 200:
            result = response.json()
            print(result)  # 打印返回的完整数据
            if result['status'] == '1' and result.get('pois'):
                return result['pois'][0].get('type', '未知')
    return '未知'

# 扩展处理过程
def process_addresses(df, api_key, max_addresses=20):
    poi_categories = []  # 用于存储POI分类结果

    for i, address in enumerate(df["取车地址"][:max_addresses]):  # 只处理前20个地址
        print(f"正在处理第 {i+1} 个地址：{address}")
        
        # 获取经纬度
        location = geocode_address(address, api_key)
        if location:
            # 获取POI类别
            poi_category = get_poi_category(location, api_key)
        else:
            poi_category = "无法识别"
        
        # 将结果添加到列表
        poi_categories.append(poi_category)
        
        # 延迟1秒，避免过快请求导致的连接问题
        time.sleep(1)

    # 将POI分类结果添加到DataFrame的新列中
    df['POI类别'] = pd.Series(poi_categories, index=df.index[:max_addresses])
    return df

# 处理并保存结果
df_processed = process_addresses(df, API_KEY, max_addresses=20)

# 保存处理后的结果
output_path = "D:/文档/WeChat Files/WeChat Files/wxid_ujzkebqpjc3v22/FileStorage/File/2024-11/处理后的订单(3).xlsx"
df_processed.to_excel(output_path, index=False)

print(f"处理完成，结果已保存到：{output_path}")

正在处理第 1 个地址：华亭国际-41号楼
{'suggestion': {'keywords': [], 'cities': []}, 'count': '344', 'infocode': '10000', 'pois': [{'parent': [], 'address': '海光路与向阳道交叉口东北300米', 'distance': '8', 'pname': '天津市', 'importance': [], 'biz_ext': {'cost': [], 'rating': '4.0'}, 'biz_type': [], 'cityname': '天津市', 'type': '生活服务;物流速递;物流速递', 'photos': [{'title': [], 'url': 'http://store.is.autonavi.com/showpic/006e75c70591e8438f4946d81a0d70fc'}], 'typecode': '070500', 'shopinfo': '0', 'poiweight': [], 'childtype': [], 'adname': '西青区', 'name': '妈妈驿站', 'location': '117.071077,39.119139', 'tel': [], 'shopid': [], 'id': 'B0JRL75QFV'}, {'parent': 'B00160DVFV', 'address': '海光路与向阳道交叉口东北340米', 'distance': '48', 'pname': '天津市', 'importance': [], 'biz_ext': {'cost': [], 'opentime2': [], 'rating': '2.3', 'open_time': [], 'meal_ordering': '0'}, 'biz_type': 'diner', 'cityname': '天津市', 'type': '餐饮服务;中餐厅;中餐厅|体育休闲服务;休闲场所;休闲场所', 'photos': [], 'typecode': '050100|080500', 'shopinfo': '0', 'poiweight': [], 'childtype': '320', 'adnam

In [17]:
import pandas as pd
import requests
import time

# 高德地图 API 密钥
API_KEY = "636216d58bb89b4b57b38e03837fc8c6"

# 读取 Excel 文件
file_path = r"D:\课程\数据分析\租车\提取后的11月订单数据.xlsx"
df = pd.read_excel(file_path)

# 定义函数获取经纬度
def geocode_address(address, api_key):
    url = f'https://restapi.amap.com/v3/geocode/geo?address={address}&key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        if result['status'] == '1' and result.get('geocodes'):
            return result['geocodes'][0]['location']  # 返回经纬度
    return None

# 获取POI类别，并优先判断是否属于大学
def get_poi_category_with_university(location, api_key, radius=500, university_keywords=("大学", "学院", "教育", "校区", "研究所")):
    if location:
        lat, lon = location.split(',')
        url = f'https://restapi.amap.com/v3/place/around?location={lon},{lat}&key={api_key}&radius={radius}'
        
        # 打印请求的URL
        print(f"请求 URL: {url}")
        
        response = requests.get(url)
        
        if response.status_code == 200:
            result = response.json()
            
            # 打印返回的完整结果
            print(f"返回结果: {result}")
            
            if result['status'] == '1' and result.get('pois'):
                pois = result['pois']
                
                # 打印POI列表
                print(f"周边 POI 列表 ({len(pois)} 个):")
                for poi in pois:
                    print(f"  - 名称: {poi.get('name', '未知')}, 类型: {poi.get('type', '未知')}")
                
                # 优先判断是否存在大学相关POI
                for poi in pois:
                    if any(keyword in poi['name'] or keyword in poi.get('type', '') for keyword in university_keywords):
                        return "大学"
                
                # 如果没有找到大学相关POI，返回第一个POI类别
                return pois[0].get('type', '未知')
    return '未知'

# 数据处理过程，直接覆盖已有的“POI类别”列
def process_addresses_with_university(df, api_key, max_addresses=20):
    poi_categories = []  # 用于存储POI分类结果

    for i, address in enumerate(df["取车地址"][:max_addresses]):  # 只处理前 max_addresses 个地址
        print(f"正在处理第 {i+1} 个地址：{address}")
        
        # 获取经纬度
        location = geocode_address(address, api_key)
        if location:
            # 获取POI类别，增加大学识别逻辑
            poi_category = get_poi_category_with_university(location, api_key)
        else:
            poi_category = "无法识别"
        
        # 将结果添加到列表
        poi_categories.append(poi_category)
        
        # 延迟1秒，避免过快请求导致的连接问题
        time.sleep(1)

    # 覆盖已有的“POI类别”列
    df.loc[df.index[:max_addresses], 'POI类别'] = poi_categories
    return df

# 处理并保存结果
df_processed = process_addresses_with_university(df, API_KEY, max_addresses=20)

# 保存处理后的结果，避免覆盖原文件
output_path = r"D:\课程\数据分析\租车\提取后的11月订单数据_处理后.xlsx"
df_processed.to_excel(output_path, index=False)

print(f"处理完成，结果已保存到：{output_path}")

正在处理第 1 个地址：张园
请求 URL: https://restapi.amap.com/v3/place/around?location=37.605799,105.829215&key=636216d58bb89b4b57b38e03837fc8c6&radius=500
返回结果: {'suggestion': {'keywords': [], 'cities': []}, 'count': '0', 'infocode': '10000', 'pois': [], 'status': '1', 'info': 'OK'}
正在处理第 2 个地址：溪竹苑小区4栋
请求 URL: https://restapi.amap.com/v3/place/around?location=39.126603,117.061996&key=636216d58bb89b4b57b38e03837fc8c6&radius=500
返回结果: {'suggestion': {'keywords': [], 'cities': []}, 'count': '5', 'infocode': '10000', 'pois': [{'parent': [], 'address': '中北镇雷庄', 'distance': '109', 'pname': '天津市', 'importance': [], 'biz_ext': {'cost': '11160.00', 'rating': []}, 'biz_type': [], 'cityname': '天津市', 'type': '商务住宅;住宅区;住宅小区', 'photos': [{'title': '外景图', 'url': 'http://store.is.autonavi.com/showpic/1d1655a6bfe6bb7f874a54a1ad09734f'}, {'title': [], 'url': 'http://store.is.autonavi.com/showpic/ae9cc0f42f96d12d74c8a9938bc4d743'}, {'title': '外景图', 'url': 'http://store.is.autonavi.com/showpic/745483e06ab72f3a4148d1d4

C:\Users\56242\AppData\Local\Temp\ipykernel_23084\3581108113.py:78: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['未知', '商务住宅;住宅区;住宅小区', '无法识别', '大学', '餐饮服务;咖啡厅;星巴克咖啡', '餐饮服务;咖啡厅;咖啡厅', '餐饮服务;餐饮相关场所;餐饮相关', '无法识别', '未知', '餐饮服务;中餐厅;中餐厅', '生活服务;邮局;邮局', '生活服务;美容美发店;美容美发店', '无法识别', '生活服务;售票处;公交卡/月票代售点', '商务住宅;楼宇;商务写字楼', '商务住宅;产业园区;产业园区', '餐饮服务;咖啡厅;星巴克咖啡', '生活服务;邮局;邮局', '无法识别', '大学']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[df.index[:max_addresses], 'POI类别'] = poi_categories


PermissionError: [Errno 13] Permission denied: 'D:\\课程\\数据分析\\租车\\提取后的11月订单数据_处理后.xlsx'

### 先坐标查询是否三站一场，之后POI周边搜索类别

In [19]:
import pandas as pd
import requests
import time
from geopy.distance import geodesic

# 高德地图 API 密钥
API_KEY = "636216d58bb89b4b57b38e03837fc8c6"

# 预设 POI 坐标及分类
preset_poi = {
    "天津站": (39.137783, 117.211693),
    "天津西站": (39.165834, 117.163548),
    "天津南站": (39.057057, 117.060761),
    "天津滨海国际机场": (39.126976, 117.362141)
}

# 定义函数获取经纬度
def geocode_address(address, api_key):
    url = f'https://restapi.amap.com/v3/geocode/geo?address={address}&key={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        if result['status'] == '1' and result.get('geocodes'):
            return result['geocodes'][0]['location']  # 返回经纬度
    return None

# 计算地址与预设 POI 的距离
def get_preset_poi_category(location):
    loc1 = tuple(map(float, location.split(',')))
    for poi_name, poi_coords in preset_poi.items():
        distance = geodesic(loc1, poi_coords).km  # 计算与 POI 的距离（千米）
        if poi_name in ["天津站", "天津西站", "天津南站"] and distance <= 1.2:
            return poi_name
        elif poi_name == "天津滨海国际机场" and distance <= 5.2:
            return "天津滨海国际机场"
    return None  # 如果未找到匹配的 POI，返回 None

# 获取 POI 类别，并优先判断是否属于大学
def get_poi_category_with_university(location, api_key, radius=500, university_keywords=("大学", "学院", "教育", "校区", "研究所")):
    lat, lon = location.split(',')
    url = f'https://restapi.amap.com/v3/place/around?location={lon},{lat}&key={api_key}&radius={radius}'
    
    response = requests.get(url)
    if response.status_code == 200:
        result = response.json()
        if result['status'] == '1' and result.get('pois'):
            pois = result['pois']
            
            # 优先判断是否存在大学相关 POI
            for poi in pois:
                if any(keyword in poi['name'] or keyword in poi.get('type', '') for keyword in university_keywords):
                    return "大学"
            
            # 如果没有找到大学相关 POI，返回第一个 POI 类别
            return pois[0].get('type', '未知')
    return '未知'

# 数据处理过程，直接覆盖已有的“POI类别”列
def process_addresses_with_university(df, api_key, max_addresses=20):
    poi_categories = []  # 用于存储 POI 分类结果

    for i, address in enumerate(df["取车地址"][:max_addresses]):  # 只处理前 max_addresses 个地址
        print(f"正在处理第 {i+1} 个地址：{address}")
        
        # 获取经纬度
        location = geocode_address(address, api_key)
        if location:
            # 优先判断是否属于预设 POI 范围
            preset_poi_category = get_preset_poi_category(location)
            if preset_poi_category:
                poi_category = preset_poi_category
            else:
                # 如果不在预设范围，获取 POI 类别
                poi_category = get_poi_category_with_university(location, api_key)
        else:
            poi_category = "无法识别"
        
        # 将结果添加到列表
        poi_categories.append(poi_category)
        
        # 延迟 1 秒，避免过快请求导致连接问题
        time.sleep(1)

    # 覆盖已有的“POI类别”列
    df.loc[df.index[:max_addresses], 'POI类别'] = poi_categories
    return df

# 读取 Excel 文件
file_path = r"D:\课程\数据分析\租车\提取后的11月订单数据.xlsx"
df = pd.read_excel(file_path)

# 处理并保存结果
df_processed = process_addresses_with_university(df, API_KEY, max_addresses=20)

# 保存处理后的结果，避免覆盖原文件
output_path = r"D:\课程\数据分析\租车\提取后的11月订单数据_处理后.xlsx"
df_processed.to_excel(output_path, index=False)

print(f"处理完成，结果已保存到：{output_path}")

处理完成，结果已保存到：D:\课程\数据分析\租车\提取后的11月订单数据_处理后.xlsx


In [ ]:
import pandas as pd
import requests
import time

# 高德地图 API 密钥
API_KEY = "ace4cdbddb2061c8b5684b043aa1ed8e"

# 读取 Excel 文件
file_path = r"D:\课程\数据分析\租车\提取后的11月订单数据.xlsx"
df = pd.read_excel(file_path)

# 使用 Geocoding API 获取地址的坐标
def geocode_address(address, api_key):
    url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={api_key}"
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if data["status"] == "OK":
                location = data["results"][0]["geometry"]["location"]
                return f"{location['lat']},{location['lng']}"  # 返回纬度和经度
            else:
                print(f"地址 {address} 的返回状态：{data['status']}")
        else:
            print(f"请求失败，HTTP 状态码：{response.status_code}")
    except Exception as e:
        print(f"获取地址 {address} 时出现错误：{e}")
    return None

# 获取预设 POI 类别（例如天津站、天津西站等）
def get_preset_poi_category(location):
    # 预设 POI 坐标和类别
    preset_pois = [
        ("天津站", "117.211693", "39.137783", 1.2),
        ("天津西站", "117.163548", "39.165834", 1.2),
        ("天津南站", "117.060761", "39.057057", 1.2),
        ("天津滨海国际机场", "117.362141", "39.126976", 5.2),
    ]
    
    if location:
        lon, lat = map(float, location.split(","))
        for name, preset_lon, preset_lat, radius in preset_pois:
            distance = ((lon - float(preset_lon)) ** 2 + (lat - float(preset_lat)) ** 2) ** 0.5 * 111  # 计算距离（近似）
            if distance <= radius:
                return name
    return None

# 获取 POI 类别，并优先判断是否属于大学
def get_poi_category_with_university(location, api_key, radius=500, university_keywords=("大学", "学院", "教育", "校区", "研究所")):
    if location:
        lat, lon = location.split(',')
        url = f'https://restapi.amap.com/v3/place/around?location={lon},{lat}&key={api_key}&radius={radius}'
        response = requests.get(url)
        if response.status_code == 200:
            result = response.json()
            if result['status'] == '1' and result.get('pois'):
                pois = result['pois']
                for poi in pois:
                    if any(keyword in poi['name'] or keyword in poi.get('type', '') for keyword in university_keywords):
                        return "大学"
                return pois[0].get('type', '未知')
    return '未知'

# 提取所有地址对应的坐标（带重试机制）
def extract_coordinates(df, api_key, retries=3, delay=1):
    coordinates = []
    for i, address in enumerate(df["取车地址"]):
        print(f"正在获取第 {i+1} 个地址的坐标：{address}")
        attempt = 0
        location = None
        while attempt < retries and location is None:
            attempt += 1
            location = geocode_address(address, api_key)
            if location:
                break
            print(f"第 {attempt} 次尝试失败，等待 {delay} 秒后重试...")
            time.sleep(delay)
            delay *= 2  # 每次重试后增加等待时间
        coordinates.append(location if location else "无法获取")
    df['坐标'] = coordinates
    return df

# 针对前 max_addresses 个数据进行 POI 类别分类
def process_addresses_with_university(df, api_key, max_addresses=2):
    poi_categories = []
    for i, location in enumerate(df["坐标"][:max_addresses]):
        print(f"正在处理第 {i+1} 个坐标：{location}")
        if location and location != "无法获取":
            # 优先判断是否属于预设 POI 范围
            preset_poi_category = get_preset_poi_category(location)
            if preset_poi_category:
                poi_category = preset_poi_category
            else:
                # 如果不在预设范围，获取 POI 类别
                poi_category = get_poi_category_with_university(location, api_key)
        else:
            poi_category = "无法识别"
        poi_categories.append(poi_category)
        time.sleep(1)
    df.loc[df.index[:max_addresses], 'POI类别'] = poi_categories
    return df

# 主流程：提取坐标并进行 POI 分类
df_with_coordinates = extract_coordinates(df, API_KEY)  # 提取所有坐标
df_processed = process_addresses_with_university(df_with_coordinates, API_KEY, max_addresses=2)  # 分类前 20 个

# 保存处理后的结果
output_path = r"D:\课程\数据分析\租车\提取后的11月订单数据_处理后.xlsx"
df_processed.to_excel(output_path, index=False)
print(f"处理完成，结果已保存到：{output_path}")

In [45]:
# 读取 Excel 文件
file_path = "D:/文档/WeChat Files/WeChat Files/wxid_ujzkebqpjc3v22/FileStorage/File/2024-11/处理后的订单(3).xlsx"
df = pd.read_excel(file_path)
df_filtered = df[['承租人', '网络平台订单号', '取车地址', 'POI类别']]
df.head(20)

,承租人,网络平台订单号,所属门店,车牌号,品牌车型,订单状态,业务负责人,发车员,收车员,油表起始,...,取车油表,取车公里数,还车公里数,还车油表,取车核实状态,取车核实备注,还车核实状态,还车核实备注,车辆供应商,POI类别
0,刘昕鑫,1128134455008194,永安租车,津AAC7569,海豚,待用车,王显生,未分配,未分配,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,自有,生活服务;物流速递;物流速递
1,程海鹰,1128134454913913,永安租车,津AFE1607,帕萨特,用车中,王显生,赵帅,未分配,16/16,...,16/16,52988.0,NaN,NaN,NaN,NaN,NaN,NaN,自有,生活服务;生活服务场所;生活服务场所
2,张子豪,2373280322361330966,永安租车,津AG21819,秦L,待用车,未分配,未分配,未分配,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,自有,商务住宅;楼宇;商务写字楼
3,保养,dz2411131208009394,永安租车,津AG27186,秦L,待用车,王显生,未分配,未分配,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,自有,无法识别
4,牛易豪,1128134454437181,永安租车,津AG27118,秦L,用车中,王显生,李彬云,未分配,16/16,...,16/16,5982.0,NaN,NaN,NaN,NaN,NaN,NaN,自有,商务住宅;住宅区;住宅小区
5,郑哲勇,1128134454557846,永安租车,津AG25100,秦L,用车中,王显生,李昕皓,未分配,16/16,...,16/16,4353.0,NaN,NaN,NaN,NaN,NaN,NaN,自有,生活服务;彩票彩券销售点;彩票彩券销售点
6,梁利君,RC202411167282196659766128730,永安租车,津AAC0777,海豚,用车中,王显生,赵帅,赵帅,9/16,...,9/16,39151.0,NaN,NaN,NaN,NaN,NaN,NaN,自有,商务住宅;住宅区;住宅小区
7,史勤华,1128134444846051,永安租车,津F8G517,卡罗拉,用车中,王显生,李新宇,未分配,16/16,...,16/16,68450.0,NaN,NaN,NaN,NaN,NaN,NaN,自有,餐饮服务;中餐厅;中餐厅
8,张永好,4129143843490266527,永安租车,津AAU2113,海豚,待用车,未分配,金洋,未分配,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,自有,商务住宅;住宅区;住宅小区
9,马思远,RC202411127281477454606805017,永安租车,津F73R41,科沃兹,待用车,王显生,未分配,未分配,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,自有,餐饮服务;糕饼店;糕饼店


### 1.对于高校和三站一场有更敏感才行

2.周边搜索日配额100

3.表格先剔除无关列